In [3]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

get_ipython().magic('matplotlib inline')
get_ipython().magic('reload_ext autoreload')
get_ipython().magic('autoreload 2')

## Unit Testing

In [ ]:
from streetstyle_dataset import StreetStyleDataset
import data_utils

transform = transforms.Compose([
        data_utils.ResizeTransform(299),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
dataset = StreetStyleDataset('../data/streetstyle27k', '../data/', transform=transform)

def imshow(inp):
    """Imshow for Tensor."""
    plt.figure()
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)

images, labels = dataset.next_train()
img = torchvision.utils.make_grid(images[:4], nrow=2)
imshow(img)

plt.figure()
images, labels = dataset.next_eval()
img = torchvision.utils.make_grid(images[:4], nrow=2)
imshow(img)

images, labels = dataset.next_test()
img = torchvision.utils.make_grid(images[:4], nrow=2)
imshow(img)

In [ ]:
from classifier_model import StreetStyleClassifier
from torch.autograd import Variable

imag_var = Variable(images)
model = StreetStyleClassifier()
model.set_eval_attributes([True, True, False, True, False, False,False,False,False,False,False,False])
out = model(imag_var)
print(out)

# Use our trained best model to infer the unknow images

In [1]:
import torch
import torch.nn as nn
import torchvision
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pickle

get_ipython().magic('matplotlib inline')
get_ipython().magic('reload_ext autoreload')
get_ipython().magic('autoreload 2')

from newsanchor_classifier_infer import NewsAnchorClassifierInfer

In [2]:
test = NewsAnchorClassifierInfer(use_gpu=True)

In [ ]:
test.create_data_loaders('../../data/cloth/cloth_all_img', '../../data/cloth/cloth_manifest.pkl', batch_size=64)
# test.create_data_loaders('../data/cloth_test_img', '../data/cloth_test_manifest.pkl', batch_size=64)

In [ ]:
test.visualize_single_batch()

In [ ]:
def get_num_parameters(model):
    total_num=0
    if type(model) == type(dict()):
        for key in model:
            for p in model[key].parameters():
                total_num+=p.nelement()
    else:
        for p in model.parameters():
            total_num+=p.nelement()
    return total_num

test.load_model("trained_models/newsanchor_850iters_loss_1-3lr_earlystopping.tar")
print(test.model)
print('num params: ' + str(get_num_parameters(test.model)))

In [ ]:
infer_result, feature_result = test.infer()

In [12]:
pickle.dump(infer_result, open('../../data/cloth/cloth_all_infer.pkl', 'wb'), protocol=2)
pickle.dump(feature_result, open('../../data/cloth/cloth_all_feature.pkl', 'wb'), protocol=2)

# Visualize result

In [2]:
from utility import *
import cv2
import pickle 
import os
import random

In [3]:
result = pickle.load(open('../../data/cloth/cloth_all_infer.pkl', 'rb'))

In [9]:
## create structure of [[manifest per anchor]] 
pid = -1
current_video = ''
current_anchor = 0
cloth_infer_group = []
for idx, p in enumerate(result):
    video = p[0]
    anchor_id = p[1]
    if video != current_video or anchor_id != current_anchor:
        cloth_infer_group.append([])
        current_video = video
        current_anchor = anchor_id
    cloth_infer_group[-1].append(p)

In [11]:
sample = random.sample(cloth_infer_group, 200)
result_vis = []
for res in sample:
    for p in res:
        result_vis.append(p)

In [13]:
attri_name = ['pattern', 'major_color', 'necktie', 'collar', 'scarf', 'sleeve', 'neckline', 'clothing', 'jacket', 'hat', \
              'glass', 'layer', 'necktie_color', 'necktie_pattern', 'hair_color', 'hair_length']
attri_dict = [
        {'solid': 0, 'graphics' : 1, 'striped' : 2, 'floral' : 3, 'plaid' : 4, 'spotted' : 5}, # clothing_pattern
        {'black' : 0, 'white' : 1, 'more color' : 2, 'blue' : 3, 'gray' : 4, 'red' : 5,
                'pink' : 6, 'green' : 7, 'yellow' : 8, 'brown' : 9, 'purple' : 10, 'orange' : 11,
                'cyan' : 12, 'dark blue' : 13}, # major_color
        {'necktie no': 0, 'necktie yes' : 1}, # wearing_necktie
        {'collar no': 0, 'collar yes' : 1}, # collar_presence
        {'scarf no': 0, 'scarf yes' : 1}, # wearing_scarf
        {'long sleeve' : 0, 'short sleeve' : 1, 'no sleeve' : 2}, # sleeve_length
        {'round' : 0, 'folded' : 1, 'v-shape' : 2}, # neckline_shape
        {'shirt' : 0, 'outerwear' : 1, 't-shirt' : 2, 'dress' : 3,
            'tank top' : 4, 'suit' : 5, 'sweater' : 6}, # clothing_category
        {'jacket no': 0, 'jacket yes' : 1}, # wearing_jacket
        {'hat no': 0, 'hat yes' : 1}, # wearing_hat
        {'glasses no': 0, 'glasses yes' : 1}, # wearing_glasses
        {'one layer': 0, 'more layer' : 1}, # multiple_layers
        {'black' : 0, 'white' : 1, 'more color' : 2, 'blue' : 3, 'gray' : 4, 'red' : 5,
                'pink' : 6, 'green' : 7, 'yellow' : 8, 'brown' : 9, 'purple' : 10, 'orange' : 11,
                'cyan' : 12, 'dark blue' : 13}, # necktie_color
        {'solid' : 0, 'striped' : 1, 'spotted' : 2}, # necktie_pattern
        {'black' : 0, 'white': 1, 'blond' : 2, 'brown' : 3, 'gray' : 4}, # hair_color
        {'long' : 0, 'medium' : 1, 'short' : 2, 'bald' : 3} # hair_longth
]
attri_num = len(attri_name)
attri_dict_inv = []
for d in attri_dict:
    d_inv = {}
    for k, v in d.items():
        d_inv[v] = k
    attri_dict_inv.append(d_inv)

In [ ]:
from skimage.util import view_as_blocks

def view_grid(im_in,ncols=3):
    n,h,w,c = im_in.shape
    dn = (-n)%ncols # trailing images
    im_out = (np.empty((n+dn)*h*w*c,im_in.dtype)
           .reshape(-1,w*ncols,c))
    view=view_as_blocks(im_out,(h,w,c))
    for k,im in enumerate( list(im_in) + dn*[0] ):
        view[k//ncols,k%ncols,0] = im 
    return im_out

GRID_W = 100
GRID_H = 200
img_data_dir = '../../data/cloth/cloth_all_img/'

images = []
cnt = 0
for res in result_vis:
    img = cv2.imread(os.path.join(img_data_dir, res[2]))
    img_desp = np.zeros((GRID_H, GRID_W, 3))
    row_height = int(GRID_H / (attri_num+2))
    text = 'id: ' + str(cnt)
    cnt += 1
    cv2.putText(img_desp, text, (0, row_height), cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.3, color=(255,255,255), thickness=1)
    for idx, c in enumerate(res[-1]):
        text = attri_name[idx] + ' : ' + attri_dict_inv[idx][c]
        cv2.putText(img_desp, text, (0, row_height*(idx+2)), cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.3, color=(255,255,255), thickness=1)
    images.append(img)
    images.append(img_desp)
    
images_np = np.zeros((len(images), GRID_H, GRID_W, 3))
for i, im in enumerate(images):
    if i % 2:
        images_np[i] = im
        continue
        
    H, W, C = im.shape
    if 1. * H / W <= 2: 
        w = GRID_W
        h = int(1. * H / W * w) 
    else:
        h = GRID_H
        w = int(1. * W / H * h)
    im_regular = cv2.resize(im, (w, h))
    images_np[i, :h, :w, :] = im_regular
    
grid = view_grid(images_np, 10)
# H, W, C = grid.shape
filename = '../data/cloth_test1.jpg'
# grid_small = cv2.resize(grid, (WIDTH*ncol, int(H/W*WIDTH*ncol)))
cv2.imwrite(filename, grid)